In [1]:
from LAVIS.lavis.datasets.builders import load_dataset
import torch
from LAVIS.lavis.models import load_model_and_preprocess

device = torch.device("cuda")
model, vis_processors, txt_processors = load_model_and_preprocess(name="blip2_t5", model_type="pretrain_flant5xl", is_eval=True, device=device)


/home/chuyu/.pyenv/versions/3.10.13/envs/blip2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/chuyu/.pyenv/versions/3.10.13/envs/blip2/lib/python3.10/site-packages/diffusers/models/cross_attention.py:30: FutureWarning: Importing from cross_attention is deprecated. Please import from diffusers.models.attention_processor instead.
  deprecate(
Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.61s/it]


In [3]:
vqa_raw_dataset = load_dataset("science_qa")
vqa_raw_dataset = list(vqa_raw_dataset["test"])
answer = []
gt = []
failed_count = 0
batch_size = 8

for i in range(0, len(vqa_raw_dataset), batch_size):
    questions = [data["question"] for data in vqa_raw_dataset[i:i+batch_size]]
    images = torch.stack([vis_processors["eval"](data["image"]) for data in vqa_raw_dataset[i:i+batch_size]], dim=0).to(device)
    text_input = [(data["context"], data["question"], data["choices"]) for data in vqa_raw_dataset[i:i+batch_size]]
    prompt = "Context: {} Question: {} Options: {} Answer: The answer is "
    predict_answer = model.predict_answers(samples={"image": images, "text_input": text_input}, prompt=prompt, inference_method="generate")
    print(predict_answer)

['(B)', '(B)', '(A)', 'C', '(C)', '(C)', '(D)', '(B)']
['(B)', '(C)', '(A)', '(B)', '(A)', '(D)', '(D)', '(D)']
['(B)', '(B)', '(B)', '(C)', '(B)', '(B)', '(A)', '(A)']
['A', '(C)', '(B)', '(A)', '(A)', '(B)', '(A)', '(A)']
['(C)', '(B)', '(A)', '(C)', '(A)', '(B)', '(D)', '(B)']
['(A)', '(A)', '(D)', '(B)', '(B)', '(B)', '(C)', '(C)']
['Solution A', '(B)', '(A)', '(C)', '(D)', '(C)', '(B)', '(A)']
['(B)', '(C)', '(D)', 'C', '(B)', '(A)', '(A)', '(B)']
['(C)', '(B)', '(B)', '(C)', '(B)', '(B)', '(C)', '(B)']
['(A)', '(A)', '(B)', '(C)', '(B)', '(B)', '(B)', '(C)']
['(B)', '(D)', '(C)', '(B)', 'D', 'attract', '(C)', '(A)']
['(C)', '(D)', '(D)', '(B)', '(B)', '(B)', 'attract', '(B)']
['Solution B', '(A)', '(A)', '(C)', '(C)', 'Solution A', '(A)', '(C)']
['(B)', '(A)', '(A)', '(A)', '(B)', '(C)', 'sample A', '(B)']
['(A)', '(B)', '(B)', '(D)', '(C)', '(C)', '(B)', '(B)']
['(C)', '(B)', '(B)', '(D)', '(A)', '(D)', '(A)', '(B)']
['(D)', '(B)', '(B)', '(A)', '(A)', '(A)', '(C)', '(D)']
['(C)

In [3]:
vqa_raw_dataset["test"][0]

{'image': <PIL.Image.Image image mode=RGB size=302x232>,
 'question': "Which of the following could Gordon's test show?",
 'question_id': '5',
 'context': "People can use the engineering-design process to develop solutions to problems. One step in the process is testing if a potential solution meets the requirements of the design.\nThe passage below describes how the engineering-design process was used to test a solution to a problem. Read the passage. Then answer the question below.\n\nGordon was an aerospace engineer who was developing a parachute for a spacecraft that would land on Mars. He needed to add a vent at the center of the parachute so the spacecraft would land smoothly. However, the spacecraft would have to travel at a high speed before landing. If the vent was too big or too small, the parachute might swing wildly at this speed. The movement could damage the spacecraft.\nSo, to help decide how big the vent should be, Gordon put a parachute with a 1 m vent in a wind tunnel